<img src="./pictures/DroneApp_logo.png" style="float:right; max-width: 180px; display: inline" alt="INSA" />
<img src="./pictures/logo_sizinglab.png" style="float:right; max-width: 100px; display: inline" alt="INSA" />

# Frame design



The objective of this study, is to optimize the overall design in terms of mass. For this target, the frame will be sized to withstand the resulting loads of two sizing scenarios: the **maximum take-off thrust (arms)** and a **landing with an impact speed of 1m/s (body,arms, landing gears)**. Due to the great diversity of existing models of drones in the
market, a simple design of quad-copter was considered for further calculations and steps

**Scipy** and **math** packages will be used for this notebook in order to illustrate the optimization algorithms of python.

In [15]:
import scipy
import scipy.optimize
from math import pi
from math import sqrt
from math import sin,cos,tan
import math
import numpy as np
import timeit
import pandas as pd

import ipywidgets as widgets
from ipywidgets import interactive

from IPython.display import display, HTML
pd.options.display.float_format = '{:,.2f}'.format

#### Frame drawing

*Simplified design of the drone frame and nomenclature of geometrical parameters used.*
<img src="./img/FrameDesign.jpg" alt="4-arms drone structure" width="800"/>

## Sizing scenarios

### Take-Off scenario

A maximum force produced at the take-off $F_{TO}$ generates a bending moment $M_{TO}$ equivalent to:
$M_{TO}=\frac{F_{TO}\cdot L_{arm}}{N_{arms}}$

The maximum stress $\sigma_{max}$ for a beam of rectangular cross-section is estimated with safety coefficient $k_s$ as:
$\displaystyle\sigma_{max}=\frac{H_{arm}}{2} \frac{12 \cdot Thrust \cdot l_{arm}}{H_{arm}^4-(H_{arm}-2e)^4} \leq \frac{\sigma_{alloy}}{k_s}$

which can be written with dimensionless arm aspect ratio $\pi_{arm}=\frac{e}{H_{arm}}$:
$\displaystyle H_{arm}\geq \left ( \frac{6 \cdot Thrust \cdot l_{arm} \cdot k_s}{\sigma_{alloy}(1-(1-2 \cdot \pi_{arm})^4)} \right )^{\frac{1}{3}}$

### Crash sizing scenario
The crash sizing scenario considers a maximum speed $V_{impact}$ of the drone when hitting the ground. At such speed the structure should resist (i.e. the maximum stress should not be exceeded) and for higher speeds, the landing
gears are the parts that break as structural fuses.
To calculate the equivalent maximum load resisted by the landing gears, the energy conservation law applies the kinetic energy stored in drone mass to potential energy in structural parts transitory deformation:
\begin{equation}
\begin{gathered}
\frac{1}{2}k_{eq} \cdot \delta x^2= \frac{1}{2} M_{tot} \cdot V_{impact}^2 \\
\Rightarrow F_{max} =\frac{1}{4}( k_{eq} \cdot \delta x + M_{total} \cdot g)=\frac{1}{4}(V_{impact} \cdot \sqrt{k_{eq}M_{total}} + M_{total} \cdot g)
\end{gathered}
\end{equation}

To calculate the maximum stress induced by the maximum load $F_{max}$ applied to one landing gear, the equivalent stiffness $k_{eq}$ should be determined. For this purpose, the problem is broken down into simpler structural parts and the equivalent stiffness $k_{eq}$ is expressed considering the effect of each stiffness on the whole part.

\begin{equation}
k_{eq} = 4 \cdot \frac{\overset{\sim}{k_1} \cdot \overset{\sim}{k_2}}{\overset{\sim}{k_1}+\overset{\sim}{k_2}}
\end{equation}

*Equivalent stiffness problem decomposition.*
<img src="./img/crash.jpg" alt="Equivalent stiffness problem" width="800"/>

## Sizing Code
The set of equations of a sizing code can generate typical issues such : 
- Underconstrained set of equations: the lacking equations can come from additional scenarios, estimation models or additional sizing variable. 
- overconstrained equations often due to the selection of a component on multiple critera: the adding of over-sizing coefficients and constraints in the optimization problem can generally fix this issue 
- algebraic loops often due to selection criteria requiring informations generally available after the selection 

**Underconstraint singularities** Example: two variables in one equation:

- Equation: cross section side of a beam resisting a normal stress: $\displaystyle H=\sqrt[3]{\frac{6*M_{to}}{\sigma_{bc}*(1-(1-2*T)^4)}}$

- Variables: thickness ($T$), cross section side ($H$)

- Geometrical restriction:$\displaystyle T<H$ 

- Strategy: $\displaystyle T=k_{TH}*H$ where 0<$k_{TH}$<1

The equation is thus transformed into an inequality and through a large number of iterations the value of both variables can be estimated.
 $\displaystyle H>\sqrt[3]{\frac{6*M_{to}}{\sigma_{bc}*(1-(1-2*k_{TH})^4)}}$

**Algebraic loop** : beta and Hlg to fulfill objective and contraints.

The final optimization problem depends thus of these parameters:

- $k_{TH}$: aspect ratio : ratio thickness (T) / side of the beam (H) < 1. Underconstraint 
- $k_{BH}$ aspect ratio : ratio body height (Hbody)/ height beam (H) > 1. Underconstraint
- $ \theta$ landing gear angle (0 is vertical beam) 0<Teta<90. Algebraic Loop
- $k_{TT}$ ratio landing gear thickness ( body side dimensions). Underconstraint
- $k_{L}$ aspect ratio: Length body(Lbody)/length arm (Larm). Underconstraint
- $Hlg$: Height of landing gear (space for battery or sensors). Algebraic Loop

The sizing code is defined here in a function which can give:

- an evaluation of the objective: here the frame mass
- an evaluation of the constraints: here the normal stress at the landing gear and body core, battery dimensions.

**Restrictions applied**:
1. **Strength of Materials (two constraints):** the stress resisted by the components(arm, body, landing gear), $\sigma_j$ must be lower than the maximum material stress.
2. **Geometry (one constraint)**: Volume of the body must be larger than the battery one's.
3. **Geometry (one constraint)**: The landing gear must be higher than the deformation caused during the impact and a possible camera or body hanging on the drone.


## Parameters definition

### General specifications

In [16]:
# Input Geometrical dimensions
Larm=0.35 # [m] one arm length
Narm=4 # [-] arms number
VolBat=0.132*0.043*0.027 #[m^3] Volume Battery (https://www.miniplanes.fr/eflite-accu-lipo-4s-148v-3300mah-50c-prise-ec3)


# Specifications for take off
F_to=32 # [N] global drone force for the take off
M_total=2 # [kg] total drone mass

# Specifications for landing impact
v_impact=1 # [m/s] impact speed

#Payload specifications
H_camera=0.057#[m] height camera 

### Material assumptions

In [17]:
# Material properties

# for beeam and core
Ey_bc=70.3e9 # [Pa] Young modulus
Rho_bc=2700 # [kg/m^3] Volumic mass
Sigma_bc=80e6 # [Pa] Elastic strength

# for landing gear 
Ey_lg=2e9 # [Pa] Young modulus
Rho_lg=1070 # [kg/m^3] Volumic mass
Sigma_lg=39e6 # [Pa] Elastic strength



### Design assumptions (constant)

In [18]:
k_sec=4 # [-] security coefficient

### Design variable (to optimize)

In [19]:
k_TH=0.1 # [-] aspect ratio : ratio thickness (T) / side of the beam (H) < 1
k_BH=2 # [-] aspect ratio : ratio body height (Hbody)/ height beam (H) > 1
Teta=20/90*pi/2 # [rad] landing gear angle (0 is vertical beam) 0<Teta<90
k_TT=1 # [-] aspect ratio : ratio landing gear thickness (Tlg)/ thickness beam (T). > 1
k_L=0.5 # [-] aspect ratio: Length body(Lbody)/length arm (Larm)<1
Hlg=.1 # [m] Height of landing gear (space for battery or sensors)

#Vector of parameters
parameters= scipy.array((k_TH,k_BH,Teta,k_TT,k_L,Hlg))

# Optimization bounds
# k_TH,  k_BH, Theta, k_TT, k_L, H_LG
bounds = [(0.15,0.4), (1,4), (30/90*pi/2,pi/2), (1,100), (0,1), (0.01,1.165)]


<a id='#section5'></a>

In [23]:
def SizingCode(param,arg):
    #Design Variables
    k_TH=param[0]
    k_BH=param[1]
    Teta=param[2]
    k_TT=param[3]
    k_L=param[4]
    Hlg=param[5]
    
    #### Beam Sizing - Take Off

    M_to=F_to/Narm*Larm*k_sec # [N.m] Moment applied in the drone center
    

#    H=(M_to/Sigma_bc/(1-(1-2*k_TH)**4))**(1/3) # [m] Side length of the beam
    H=(6*M_to/Sigma_bc/(1-(1-2*k_TH)**4))**(1/3) # [m] Side length of the beam
    T=k_TH*H # [m] Thickness of the side beam


    #### Body and Landing gear sizing - Landing impact
    # Body stiffness calculation 
    Hbody=k_BH*H # [m] height of the body
    Ibody=1/12*((H+2*T)*Hbody**3-H*(Hbody-2*T)**3) # [m^4] Section inertia of the body
    Lbody=k_L*Larm #[m] length of the body
    K1=3*Ey_bc*Ibody/(Lbody)**3 # [N/m] equivalent stiffness of the body

    # Landing gear stiffness calculation 
    Llg=Hlg/cos(Teta) # [m] Landing gear length
    Tlg=k_TT*T # [m] landing gear thickness
    
    Ilg=1/12*(Tlg**4) # [m^4] Section inertia of the landing gear rectangular section
    K2=3*Ey_lg*Ilg/Llg**3/sin(Teta) # [N/m] equivalent stiffness of the landing gear

    # Global stiffness
    Kg=K1*K2/(K1+K2)*Narm # [N/m] global stiffness of all the arms

    # Impact force
    Fimpact= (v_impact*(Kg*M_total)**(1/2)+M_total*9.81)*k_sec # [N] Total impact force, we assume all the landing gear impact together

    # Stress calculation in the landing gear

    M_LG=Fimpact/Narm*Hlg*tan(Teta) # [N.m] Moment applied in the landing gear
    Sigma_lg_impact=M_LG*(Tlg/2)/Ilg # [Pa] Max stress in the landing gear

    # Stress calculation in the body

    M_Body=(Fimpact/Narm*Lbody+M_LG) # [N.m] Moment applied in the body
    Sigma_body_impact=M_Body*(Hbody/2)/Ibody # [Pa] Max stress in the landing gear

    # Mass calculation
    Mbeams=Narm*Larm*(H**2-(H-2*T)**2)*Rho_bc #[kg] Total beams' mass
    MLG=Narm*Llg*Tlg**2*Rho_lg #[kg] Total landing gears' mass
    Mbody=Narm*(Lbody)*(Hbody*(H+2*T)-(Hbody-2*T)*H)*Rho_bc #[kg] Total body's mass

    Mframe=Mbeams+MLG+Mbody #[kg] total frame mass
    Vbody=(2*Lbody)**2*Hbody #[m^3] volume body to integer battery
    # Contraintes : stress 

    
    constraints = [(Sigma_bc-Sigma_body_impact)/Sigma_body_impact,(Sigma_lg-Sigma_lg_impact)/Sigma_lg_impact,(Vbody-VolBat)/VolBat,(Hlg-Fimpact/(Narm*Kg)-H_camera)/(Hlg)]
    # Objectif : masse totale 
    if arg=='Obj':
        return Mframe
    elif arg == 'ObjP':
        P = 0. # Penalisation nulle
        for C in constraints: 
            if (C < 0.): 
                P = P-1e9*C
        return Mframe + P #mass optimizatin

    elif arg=='Prt':
        col_names_opt = ['Type', 'Name', 'Min', 'Value', 'Max', 'Unit', 'Comment']

        df_opt = pd.DataFrame()
        df_opt = df_opt.append([{'Type': 'Optimization', 'Name': 'k_TH', 'Min': bounds[0][0], 'Value': k_TH, 'Max': bounds[0][1], 'Unit': '[-]', 'Comment': 'Aspect ratio for the beam\'s thickness (T/H), '}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Optimization', 'Name': 'k_BH', 'Min': bounds[1][0], 'Value': k_BH, 'Max': bounds[1][1], 'Unit': '[-]', 'Comment': 'Aspect ratio for the body\'s height (Hbody/H)'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Optimization', 'Name': 'Theta', 'Min': bounds[2][0], 'Value': Teta/pi*180, 'Max': bounds[2][1], 'Unit': '[-]', 'Comment': 'Angle of the landing gear w.r.t. the beam'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Optimization', 'Name': 'k_TT', 'Min': bounds[3][0], 'Value': k_TT, 'Max': bounds[3][1], 'Unit': '[-]', 'Comment': 'Aspect ratio for the Landing gear\'s thickness (Tlg/T)'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Optimization', 'Name': 'k_L', 'Min': bounds[4][0], 'Value': k_L, 'Max': bounds[4][1], 'Unit': '[-]', 'Comment': 'Aspect ratio: Length body(Lbody)/length arm (Larm) k_L'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Optimization', 'Name': 'Hlg', 'Min': bounds[5][0], 'Value': Hlg, 'Max': bounds[5][1], 'Unit': '[-]', 'Comment': 'Landing gear height'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Optimization', 'Name': 'Mbeams', 'Min': 0, 'Value': Mbeams, 'Max': '-', 'Unit': '[kg]', 'Comment': 'Total beams mass'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Optimization', 'Name': 'MLG', 'Min': 0, 'Value': MLG, 'Max': '-', 'Unit': '[kg]', 'Comment': 'Total landing gear mass'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Optimization', 'Name': 'Mbody', 'Min': 0, 'Value': Mbody, 'Max': '-', 'Unit': '[kg]', 'Comment': 'Total body mass'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Optimization', 'Name': 'Const 0', 'Min': 0, 'Value': constraints[0], 'Max': '-', 'Unit': '[-]', 'Comment': 'Stress margin at the Body: (Sigma_bc-Sigma_body_impact)/Sigma_body_impact'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Optimization', 'Name': 'Const 1', 'Min': 0, 'Value': constraints[1], 'Max': '-', 'Unit': '[-]', 'Comment': 'Stress margin at the landing gears: (Sigma_lg-Sigma_lg_impact)/Sigma_lg_impact'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Optimization', 'Name': 'Const 2', 'Min': 0, 'Value': constraints[2], 'Max': '-', 'Unit': '[-]', 'Comment': '(Vbody-VolBat)/VolBat'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Optimization', 'Name': 'Const 3', 'Min': 0, 'Value': constraints[3], 'Max': '-', 'Unit': '[-]', 'Comment': '(Hlg-Fimpact/(Narm*Kg)-H_camera)/(Hlg)'}])[col_names_opt]
 
        col_names = ['Type', 'Name', 'Value', 'Unit', 'Comment']

        df = pd.DataFrame()

        df = df.append([{'Type': 'Arm', 'Name': 'Larm', 'Value': Larm, 'Unit': '[m]', 'Comment': 'Arm length'}])[col_names]
        df = df.append([{'Type': 'Arm', 'Name': 'H', 'Value': H, 'Unit': '[m]', 'Comment': 'Height beam'}])[col_names]
        df = df.append([{'Type': 'Arm', 'Name': 'T', 'Value': T, 'Unit': '[m]', 'Comment': 'Thickness arm'}])[col_names]
        df = df.append([{'Type': 'Body', 'Name': 'Lbody', 'Value': Lbody, 'Unit': '[m]', 'Comment': 'Body length'}])[col_names]
        df = df.append([{'Type': 'Body', 'Name': 'Hbody', 'Value': Hbody, 'Unit': '[m]', 'Comment': 'Body height'}])[col_names]
        df = df.append([{'Type': 'Body', 'Name': 'H+2*T', 'Value': H+2*T, 'Unit': '[m]', 'Comment': 'Body width'}])[col_names]
        df = df.append([{'Type': 'Crash', 'Name': 'v_impact', 'Value': v_impact, 'Unit': '[m/s]', 'Comment': 'Crash speed'}])[col_names]
        df = df.append([{'Type': 'Crash', 'Name': 'Kg', 'Value': Kg, 'Unit': '[N/m]', 'Comment': 'Global stiffness'}])[col_names]
        df = df.append([{'Type': 'Crash', 'Name': 'k_sec', 'Value': k_sec, 'Unit': '[-]', 'Comment': 'Safety coef.'}])[col_names]
        df = df.append([{'Type': 'Crash', 'Name': 'Fimpact', 'Value': Fimpact, 'Unit': '[N]', 'Comment': 'Max crash load'}])[col_names]

        pd.options.display.float_format = '{:,.3f}'.format

        def view(x=''):
            #if x=='All': return display(df)
            if x=='Optimization' : return display(df_opt)
            return display(df[df['Type']==x])

        items = sorted(df['Type'].unique().tolist())+['Optimization']

        w = widgets.Select(options=items)

        return display(df,df_opt)

    else:
        return constraints


<a id='#section6'></a>

## Optimization problem


We will now use the [optimization algorithms](https://docs.scipy.org/doc/scipy/reference/optimize.html) of the Scipy package to solve and optimize the configuration. We use here the SLSQP algorithm without explicit expression of the gradient (Jacobian). A course on Multidisplinary Gradient optimization algorithms and gradient optimization algorithm is given [here](http://mdolab.engin.umich.edu/sites/default/files/Martins-MDO-course-notes.pdf):
> Joaquim R. R. A. Martins (2012). A Short Course on Multidisciplinary Design Optimization. University of Michigan

We can print of the characterisitcs of the problem before optimization with the initial vector of optimization variables:

In [24]:
# Initial characteristics before optimization 
print("-----------------------------------------------")
print("Initial characteristics before optimization :")
SizingCode(parameters,'Prt')
print("-----------------------------------------------")

-----------------------------------------------
Initial characteristics before optimization :


,Type,Name,Value,Unit,Comment
0,Arm,Larm,0.350,[m],Arm length
0,Arm,H,0.011,[m],Height beam
0,Arm,T,0.001,[m],Thickness arm
0,Body,Lbody,0.175,[m],Body length
0,Body,Hbody,0.022,[m],Body height
0,Body,H+2*T,0.013,[m],Body width
0,Crash,v_impact,1.000,[m/s],Crash speed
0,Crash,Kg,7.764,[N/m],Global stiffness
0,Crash,k_sec,4.000,[-],Safety coef.
0,Crash,Fimpact,94.243,[N],Max crash load


,Type,Name,Min,Value,Max,Unit,Comment
0,Optimization,k_TH,0.150,0.100,0.400,[-],"Aspect ratio for the beam's thickness (T/H),"
0,Optimization,k_BH,1.000,2.000,4.000,[-],Aspect ratio for the body's height (Hbody/H)
0,Optimization,Theta,0.524,20.000,1.571,[-],Angle of the landing gear w.r.t. the beam
0,Optimization,k_TT,1.000,1.000,100.000,[-],Aspect ratio for the Landing gear's thickness ...
0,Optimization,k_L,0.000,0.500,1.000,[-],Aspect ratio: Length body(Lbody)/length arm (L...
0,Optimization,Hlg,0.010,0.100,1.165,[-],Landing gear height
0,Optimization,Mbeams,0.000,0.172,-,[kg],Total beams mass
0,Optimization,MLG,0.000,0.001,-,[kg],Total landing gear mass
0,Optimization,Mbody,0.000,0.143,-,[kg],Total body mass
0,Optimization,Const 0,0.000,6.176,-,[-],Stress margin at the Body: (Sigma_bc-Sigma_bod...


-----------------------------------------------


In [25]:
# Optimization with SLSQP algorithm
contrainte = lambda x: SizingCode(x, 'Const')
objectif = lambda x: SizingCode(x, 'Obj')
objectifP = lambda x: SizingCode(x, 'ObjP')

SLSQP = False # Optimization algorithm choice

if SLSQP == True:
    # SLSQP omptimisation
    result = scipy.optimize.fmin_slsqp(func=objectif, x0=parameters, 
                                   bounds=bounds,
                                   f_ieqcons=contrainte, iter=1500, acc=1e-12)
else:
    # Differential evolution omptimisation
    result = scipy.optimize.differential_evolution(func=objectifP,
                                   bounds=bounds,
                                   tol=1e-12)

# Final characteristics after optimization 
print("-----------------------------------------------")
print("Final characteristics after optimization :")

if SLSQP == True:
    SizingCode(result,'Obj')
    SizingCode(result, 'Prt')
else:
    SizingCode(result.x,'Obj')
    SizingCode(result.x, 'Prt')
print("-----------------------------------------------")



-----------------------------------------------
Final characteristics after optimization :


,Type,Name,Value,Unit,Comment
0,Arm,Larm,0.350,[m],Arm length
0,Arm,H,0.010,[m],Height beam
0,Arm,T,0.002,[m],Thickness arm
0,Body,Lbody,0.031,[m],Body length
0,Body,Hbody,0.039,[m],Body height
0,Body,H+2*T,0.013,[m],Body width
0,Crash,v_impact,1.000,[m/s],Crash speed
0,Crash,Kg,"884,116.522",[N/m],Global stiffness
0,Crash,k_sec,4.000,[-],Safety coef.
0,Crash,Fimpact,"5,397.477",[N],Max crash load


,Type,Name,Min,Value,Max,Unit,Comment
0,Optimization,k_TH,0.150,0.150,0.400,[-],"Aspect ratio for the beam's thickness (T/H),"
0,Optimization,k_BH,1.000,3.803,4.000,[-],Aspect ratio for the body's height (Hbody/H)
0,Optimization,Theta,0.524,30.000,1.571,[-],Angle of the landing gear w.r.t. the beam
0,Optimization,k_TT,1.000,14.129,100.000,[-],Aspect ratio for the Landing gear's thickness ...
0,Optimization,k_L,0.000,0.089,1.000,[-],Aspect ratio: Length body(Lbody)/length arm (L...
0,Optimization,Hlg,0.010,0.088,1.165,[-],Landing gear height
0,Optimization,Mbeams,0.000,0.206,-,[kg],Total beams mass
0,Optimization,MLG,0.000,0.208,-,[kg],Total landing gear mass
0,Optimization,Mbody,0.000,0.052,-,[kg],Total body mass
0,Optimization,Const 0,0.000,0.000,-,[-],Stress margin at the Body: (Sigma_bc-Sigma_bod...


-----------------------------------------------
